<a href="https://colab.research.google.com/github/galitneu/auto-eda-tool/blob/main/%D7%95%D7%A9%D7%90%D7%A8%D7%99%D7%95%D7%AA_%D7%97%D7%99%D7%A9%D7%95%D7%91_%D7%A2%D7%9D_%D7%90%D7%99%D7%A0%D7%A4%D7%9C%D7%A6%D7%99%D7%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# --- שלב 0: ייבוא ספריות וחיבור לגוגל דרייב ---
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import GridSearchCV

# חיבור סביבת העבודה לגוגל דרייב
from google.colab import drive
drive.mount('/content/drive')

# --- שלב 1: הגדרת נתיבים, טעינת הנתונים ועיבוד ראשוני ---
DRIVE_PATH = '/content/drive/MyDrive/KaggleProject/'
try:
    df_train_raw = pd.read_csv(f'{DRIVE_PATH}Train.csv', low_memory=False)
    df_valid_raw = pd.read_csv(f'{DRIVE_PATH}Valid.csv', low_memory=False)
except FileNotFoundError:
    print(f"שגיאה: ודא שהקבצים 'Train.csv' ו-'Valid.csv' נמצאים בתיקייה: {DRIVE_PATH}")
    exit()

# המרת תאריכים וסינון הנתונים לשנים 2000 ואילך
print("מבצע סינון ראשוני לשמירת נתונים משנת 2000 ואילך...")
df_train_raw['saledate'] = pd.to_datetime(df_train_raw['saledate'])
df_valid_raw['saledate'] = pd.to_datetime(df_valid_raw['saledate'])
df_train_raw = df_train_raw[df_train_raw['saledate'].dt.year >= 2000].copy()
df_valid_raw = df_valid_raw[df_valid_raw['saledate'].dt.year >= 2000].copy()

# --- שלב 1.5: התאמת מחירים לאינפלציה ---
print("\nמבצע התאמה של מחירי המכירה לאינפלציה (לערכי 2012)...")
cpi_data_real = {
    2000: 172.2, 2001: 177.1, 2002: 179.9, 2003: 184.0, 2004: 188.9, 2005: 195.3,
    2006: 201.6, 2007: 207.3, 2008: 215.3, 2009: 214.5, 2010: 218.1, 2011: 224.9, 2012: 229.6
}
ADJUSTMENT_YEAR = 2012
inflation_multiplier = {year: cpi_data_real[ADJUSTMENT_YEAR] / cpi for year, cpi in cpi_data_real.items()}
df_train_raw['SalePrice_adj'] = df_train_raw.apply(
    lambda row: row['SalePrice'] * inflation_multiplier.get(row['saledate'].year, 1), axis=1
)
# שמירת המחיר המקורי של סט האימון בצד
train_original_prices = df_train_raw[['SalePrice', 'SalePrice_adj']].copy()

# --- שלב 1.6: יצירת מאפייני זמן מתקדמים ---
print("יוצר מאפייני זמן מתקדמים...")
df_train_raw['source'] = 'train'
df_valid_raw['source'] = 'valid'
df_combined_temp = pd.concat([df_train_raw.drop(['SalePrice', 'SalePrice_adj'], axis=1), df_valid_raw], ignore_index=True)

df_combined_temp['saleYear'] = df_combined_temp['saledate'].dt.year
df_combined_temp['saleMonth'] = df_combined_temp['saledate'].dt.month
df_combined_temp['saleDay'] = df_combined_temp['saledate'].dt.day
df_combined_temp['saleWeekOfYear'] = df_combined_temp['saledate'].dt.isocalendar().week.astype(int)
df_combined_temp['saleDayOfWeek'] = df_combined_temp['saledate'].dt.dayofweek
min_date = df_combined_temp['saledate'].min()
df_combined_temp['DaysFromStart'] = (df_combined_temp['saledate'] - min_date).dt.days

df_train_raw = df_combined_temp[df_combined_temp['source'] == 'train'].drop('source', axis=1).copy()
df_valid_raw = df_combined_temp[df_combined_temp['source'] == 'valid'].drop('source', axis=1).copy()
df_train_raw = pd.concat([df_train_raw.reset_index(drop=True), train_original_prices.reset_index(drop=True)], axis=1)
print("מאפייני זמן נוצרו בהצלחה.")

# --- שלב 1.7: פיצול הבעיה - אימון מודל מגמה וחישוב שאריות ---
print("\nמבצע הפרדת מגמה וחישוב שאריות...")
trend_model = LinearRegression()
X_trend = df_train_raw[['DaysFromStart']]
y_trend = df_train_raw['SalePrice_adj']
trend_model.fit(X_trend, y_trend)
df_train_raw['SalePrice_Trend'] = trend_model.predict(df_train_raw[['DaysFromStart']])
df_valid_raw['SalePrice_Trend'] = trend_model.predict(df_valid_raw[['DaysFromStart']])
df_train_raw['SalePrice_Residual'] = df_train_raw['SalePrice_adj'] - df_train_raw['SalePrice_Trend']
print("חישוב השאריות הושלם.")

# --- שלב 2: איחוד קבצים לעיבוד אחיד ---
train_labels_residual = df_train_raw['SalePrice_Residual'].copy()
train_labels_adj_price = df_train_raw['SalePrice_adj'].copy()
df_train_proc = df_train_raw.drop(['SalePrice', 'SalePrice_adj', 'SalePrice_Trend', 'SalePrice_Residual'], axis=1)
df_train_proc['source'] = 'train'
df_valid_proc = df_valid_raw.drop(['SalePrice_Trend'], axis=1)
df_valid_proc['source'] = 'valid'
df_combined = pd.concat([df_train_proc, df_valid_proc], ignore_index=True, sort=False)
df_combined = df_combined.drop('saledate', axis=1)

# ##################################################################
# ######### שילוב הנדסת המאפיינים המקורית שלך עם התיקון #############
# ##################################################################

# --- שלב 3: הנדסת מאפיינים (Feature Engineering) ---
print("\nמבצע הנדסת מאפיינים...")
# --- שלב 3 (משודרג): הנדסת מאפיינים עם תיקון חכם ל-YearMade ---
print("\nמבצע הנדסת מאפיינים עם תיקון חכם ל-YearMade...")

# 1. חישוב שנת הייצור החציונית לכל דגם
print("מחשב שנת ייצור חציונית לכל דגם...")
year_made_by_model = df_combined.groupby('fiModelDesc')['YearMade'].median()
year_made_by_model = year_made_by_model.astype(int)

# 2. זיהוי ותיקון השורות עם YearMade=1000
rows_to_fix_idx = df_combined[df_combined['YearMade'] == 1000].index
print(f"מתקן {len(rows_to_fix_idx)} רשומות עם YearMade=1000...")
imputed_years = df_combined.loc[rows_to_fix_idx, 'fiModelDesc'].map(year_made_by_model)
global_median_year = df_combined[df_combined['YearMade'] != 1000]['YearMade'].median()
imputed_years = imputed_years.fillna(global_median_year)
df_combined.loc[rows_to_fix_idx, 'YearMade'] = imputed_years.values
print("התיקון הושלם.")

# 3. עכשיו, אחרי שהכל תוקן, מחשבים את גיל המכונה
df_combined['machineAge'] = df_combined['saleYear'] - df_combined['YearMade']


# --- שלב 4 (משודרג): טיפול בערכים חסרים עם תיקון חכם ל-MachineHoursCurrentMeter ---
print("מטפל בערכים חסרים...")

# --- טיפול חכם ב-MachineHoursCurrentMeter ---
print("מבצע תיקון חכם ל-MachineHoursCurrentMeter=0...")
# יצירת עמודת _is_missing לפני השינויים. היא תתפוס גם NaN וגם 0.
df_combined['MachineHoursCurrentMeter_is_missing'] = df_combined['MachineHoursCurrentMeter'].isnull() | (df_combined['MachineHoursCurrentMeter'] == 0)

# החלפת 0 ב-NaN כדי שנוכל למלא אותו
df_combined['MachineHoursCurrentMeter'].replace(0, np.nan, inplace=True)

# חישוב חציון שעות עבודה לפי דגם
hours_by_model = df_combined.groupby('fiModelDesc')['MachineHoursCurrentMeter'].median()

# מילוי הערכים החסרים באמצעות החציון של הדגם המתאים
# נשתמש ב-transform כדי לשמור על האינדקס המקורי של df_combined
imputed_hours = df_combined.groupby('fiModelDesc')['MachineHoursCurrentMeter'].transform(lambda x: x.fillna(x.median()))
df_combined['MachineHoursCurrentMeter'] = imputed_hours

# אם עדיין נשארו ערכים חסרים (לדגמים נדירים), נמלא בחציון הכללי
if df_combined['MachineHoursCurrentMeter'].isnull().sum() > 0:
    global_median_hours = df_combined['MachineHoursCurrentMeter'].median()
    df_combined['MachineHoursCurrentMeter'].fillna(global_median_hours, inplace=True)

df_combined['fiProductClassDesc'] = df_combined['fiProductClassDesc'].fillna('')
keywords_to_extract = ['excavator', 'dozer', 'loader', 'crawler', 'wheel', 'track']
for keyword in keywords_to_extract:
    df_combined[f'is_{keyword}'] = df_combined['fiProductClassDesc'].str.contains(keyword, case=False).astype(int)

df_combined['is_YearMade_1000'] = (df_combined['YearMade'] == 1000).astype(int)
df_combined['is_Hours_Zero'] = (df_combined['MachineHoursCurrentMeter'] == 0).astype(int)
df_combined['is_ProductGroup_5'] = (df_combined['ProductGroup'] == 5).astype(int)
df_combined['is_ProductGroup_2'] = (df_combined['ProductGroup'] == 2).astype(int)

# הגדרת הקטגוריות הבעייתיות שמצאנו בניתוח
problematic_hydraulics = [0, 11]

# יצירת מאפיין האינטראקציה החדש
df_combined['is_Year1000_and_ProblematicHydraulics'] = (
    (df_combined['YearMade'] == 1000) &
    (df_combined['Hydraulics'].isin(problematic_hydraulics))
).astype(int)

print("נוצר מאפיין אינטראקציה חדש: 'is_Year1000_and_ProblematicHydraulics'")
# --- שלב 4: טיפול בערכים חסרים (Missing Values) ---
print("מטפל בערכים חסרים...")
# טיפול בערכים מספריים חסרים
numeric_cols_missing = ['MachineHoursCurrentMeter', 'auctioneerID']
for col in numeric_cols_missing:
    if df_combined[col].isnull().sum() > 0:
        df_combined[col + '_is_missing'] = df_combined[col].isnull()
        median_val = df_combined[col].median()
        df_combined[col] = df_combined[col].fillna(median_val)

# טיפול בערכים קטגוריאליים חסרים (עם התיקון שהופך אותו לרובוסטי)
for col_name in df_combined.columns:
    if pd.api.types.is_object_dtype(df_combined[col_name]):
        df_combined[col_name] = df_combined[col_name].fillna('missing')

# --- שלב 5: המרת עמודות קטגוריאליות למספרים ---
print("ממיר עמודות קטגוריאליות למספרים...")
# עכשיו הלוגיקה המקורית שלך תעבוד בבטחה
source_col = df_combined['source'] # שמירת עמודת המקור בצד
df_combined = df_combined.drop('source', axis=1)
cols_to_drop = []

for col_name in df_combined.columns:
    if pd.api.types.is_object_dtype(df_combined[col_name]):
        num_unique_values = df_combined[col_name].nunique()
        # לוגיקת One-Hot Encoding לעמודות עם מעט ערכים
        if num_unique_values <= 5:
            dummies = pd.get_dummies(df_combined[col_name], prefix=col_name)
            df_combined = pd.concat([df_combined, dummies], axis=1)
            cols_to_drop.append(col_name)
        # לוגיקת Label Encoding לעמודות עם הרבה ערכים
        else:
            df_combined[col_name] = pd.Categorical(df_combined[col_name]).codes

df_combined = df_combined.drop(columns=cols_to_drop)
df_combined['source'] = source_col # החזרת עמודת המקור

# --- פונקציות עזר לחישוב מדדי שגיאה ---
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
def rmsle(y_true, y_pred):
    y_pred_safe = np.maximum(y_pred, 1)
    return np.sqrt(mean_squared_log_error(y_true, y_pred_safe))

# --- שלב 6: אימון ראשוני והערכה מקומית ---
df_train_processed = df_combined[df_combined['source'] == 'train'].drop('source', axis=1).copy()
df_valid_processed = df_combined[df_combined['source'] == 'valid'].drop('source', axis=1).copy()

df_train_processed['SalePrice_Residual'] = train_labels_residual.values
df_train_processed['SalePrice_adj'] = train_labels_adj_price.values
df_train_processed['SalePrice_Trend'] = df_train_raw['SalePrice_Trend'].values

val_year = 2011
train_time_split = df_train_processed[df_train_processed['saleYear'] <= val_year - 1]
val_time_split = df_train_processed[df_train_processed['saleYear'] == val_year]

X_train_time = train_time_split.drop(['SalePrice_Residual', 'SalePrice_adj', 'SalePrice_Trend'], axis=1)
y_train_time_residual = train_time_split['SalePrice_Residual']
X_val_time = val_time_split.drop(['SalePrice_Residual', 'SalePrice_adj', 'SalePrice_Trend'], axis=1)
y_val_time_actual_price = val_time_split['SalePrice_adj']
val_time_trend = val_time_split['SalePrice_Trend']

# ודא שסדר העמודות זהה בין סט האימון והבדיקה
X_val_time = X_val_time[X_train_time.columns]

print("\n--- שלב 6: אימון מודל ראשוני על שאריות (2000-2010) ---")
initial_model = RandomForestRegressor(n_jobs=-1, random_state=42, n_estimators=150, min_samples_split=5, min_samples_leaf=4, max_features=0.5, max_depth=30)
initial_model.fit(X_train_time, y_train_time_residual)

print("\n--- ביצועי המודל הראשוני (אומן על 2000-2010) ---")
initial_val_preds_residual = initial_model.predict(X_val_time)
initial_val_preds_full_price = val_time_trend + initial_val_preds_residual
initial_val_rmse = rmse(y_val_time_actual_price, initial_val_preds_full_price)
initial_val_rmsle = rmsle(y_val_time_actual_price, initial_val_preds_full_price)

print(f"סט וולידציה (Validation Set - 2011):")
print(f"\tRMSE:  ${initial_val_rmse:,.2f}")
print(f"\tRMSLE: {initial_val_rmsle:.4f}\n")

# --- שלב 7: אימון מודל סופי והערכה סופית ---
print("="*50)
print("\n--- שלב 7: אימון מודל סופי על כל שאריות האימון (2000-2011) ---")

X_full_train = df_train_processed.drop(['SalePrice_Residual', 'SalePrice_adj', 'SalePrice_Trend'], axis=1)
y_full_train_residual = df_train_processed['SalePrice_Residual']

final_model = RandomForestRegressor(n_jobs=-1, random_state=42, n_estimators=150, min_samples_split=5, min_samples_leaf=4, max_features=0.5, max_depth=30)
final_model.fit(X_full_train, y_full_train_residual)
print("אימון המודל הסופי הושלם.")

print("\n--- ביצועי המודל הסופי (אומן על 2000-2011) ---")
final_val_preds_residual = final_model.predict(X_val_time)
final_val_preds_full_price = val_time_trend + final_val_preds_residual
final_val_rmse = rmse(y_val_time_actual_price, final_val_preds_full_price)
final_val_rmsle = rmsle(y_val_time_actual_price, final_val_preds_full_price)

print(f"סט וולידציה (Validation Set - 2011):")
print(f"\tRMSE:  ${final_val_rmse:,.2f}")
print(f"\tRMSLE: {final_val_rmsle:.4f}\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
מבצע סינון ראשוני לשמירת נתונים משנת 2000 ואילך...

מבצע התאמה של מחירי המכירה לאינפלציה (לערכי 2012)...
יוצר מאפייני זמן מתקדמים...
מאפייני זמן נוצרו בהצלחה.

מבצע הפרדת מגמה וחישוב שאריות...
חישוב השאריות הושלם.

מבצע הנדסת מאפיינים...

מבצע הנדסת מאפיינים עם תיקון חכם ל-YearMade...
מחשב שנת ייצור חציונית לכל דגם...
מתקן 33186 רשומות עם YearMade=1000...
התיקון הושלם.
מטפל בערכים חסרים...
מבצע תיקון חכם ל-MachineHoursCurrentMeter=0...


/tmp/ipython-input-540504499.py:119: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_combined['MachineHoursCurrentMeter'].replace(0, np.nan, inplace=True)
/tmp/ipython-input-540504499.py:132: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

נוצר מאפיין אינטראקציה חדש: 'is_Year1000_and_ProblematicHydraulics'
מטפל בערכים חסרים...
ממיר עמודות קטגוריאליות למספרים...

--- שלב 6: אימון מודל ראשוני על שאריות (2000-2010) ---

--- ביצועי המודל הראשוני (אומן על 2000-2010) ---
סט וולידציה (Validation Set - 2011):
	RMSE:  $9,670.11
	RMSLE: 0.2406


--- שלב 7: אימון מודל סופי על כל שאריות האימון (2000-2011) ---
אימון המודל הסופי הושלם.

--- ביצועי המודל הסופי (אומן על 2000-2011) ---
סט וולידציה (Validation Set - 2011):
	RMSE:  $4,955.23
	RMSLE: 0.1363



In [17]:
# --- שלב 8: יצירת קובץ הגשה סופי עם מודל השאריות ---
print("="*50)
print("\n--- שלב 8: יצירת קובץ הגשה באמצעות המודל הסופי (מודל השאריות) ---")

# 1. התאמת עמודות בין סט האימון לסט הוולידציה (הבדיקה)
# שלב זה חיוני כדי להבטיח שלסט הוולידציה יש בדיוק את אותן עמודות כמו לסט שהמודל אומן עליו
train_cols = X_full_train.columns
valid_cols_to_align = df_valid_processed[train_cols]


# 2. חיזוי דו-שלבי
# 2.1: חיזוי השאריות באמצעות המודל המורכב
print("שלב 1/3: מבצע חיזוי לשאריות...")
final_residuals_pred = final_model.predict(valid_cols_to_align)

# 2.2: שליפת ערכי המגמה שכבר חישבנו עבור סט הוולידציה
print("שלב 2/3: משחזר את ערכי המגמה...")
final_trend_values = df_valid_raw['SalePrice_Trend'].values

# 3. שחזור המחיר הסופי
# החיזוי הסופי הוא הסכום של המגמה החזויה והשארית החזויה
print("שלב 3/3: מחבר את המגמה והשאריות ליצירת חיזוי סופי...")
final_price_pred = final_trend_values + final_residuals_pred


# 4. ליטוש אחרון: קביעת רצפה למניעת מחירים שליליים
# כפי שדיברנו, נגדיר מחיר מינימום של 1000$
final_price_pred_floored = np.maximum(final_price_pred, 1000)


# 5. יצירת קובץ ההגשה ושמירתו
df_submission = pd.DataFrame({
    'SalesID': df_valid_raw['SalesID'],
    'SalePrice': final_price_pred_floored
})

submission_filename = f'{DRIVE_PATH}submission_residuals_model.csv'
df_submission.to_csv(submission_filename, index=False)

print(f"\nקובץ ההגשה '{submission_filename}' נשמר בהצלחה!")
print("5 השורות הראשונות בקובץ ההגשה:")
print(df_submission.head())


--- שלב 8: יצירת קובץ הגשה באמצעות המודל הסופי (מודל השאריות) ---
שלב 1/3: מבצע חיזוי לשאריות...
שלב 2/3: משחזר את ערכי המגמה...
שלב 3/3: מחבר את המגמה והשאריות ליצירת חיזוי סופי...

קובץ ההגשה '/content/drive/MyDrive/KaggleProject/submission_residuals_model.csv' נשמר בהצלחה!
5 השורות הראשונות בקובץ ההגשה:
        SalesID     SalePrice
313947  1222837  55456.236620
313948  1222839  72420.602852
313949  1222841  34047.907885
313950  1222843  17541.454588
313951  1222845  42399.766207


In [16]:
# הרץ את הקוד הזה אחרי שלב 6 (אימון המודל הראשוני)
print("\n--- מנתח את השגיאות הגדולות ביותר של המודל ---")

# חישוב השגיאה המוחלטת
val_time_split['predictions'] = initial_val_preds_full_price
val_time_split['error'] = abs(val_time_split['SalePrice_adj'] - val_time_split['predictions'])

# מיון לפי השגיאה הגבוהה ביותר
worst_predictions = val_time_split.sort_values('error', ascending=False)

# הצגת 10 הטעויות הגדולות ביותר
print("10 החיזויים עם השגיאה הגבוהה ביותר:")
print(worst_predictions[['SalesID', 'SalePrice_adj', 'predictions', 'error', 'ProductGroupDesc', 'YearMade', 'MachineHoursCurrentMeter']].head(10))


--- מנתח את השגיאות הגדולות ביותר של המודל ---
10 החיזויים עם השגיאה הגבוהה ביותר:
        SalesID  SalePrice_adj   predictions         error  ProductGroupDesc  \
51489   1350808  107194.308582  27661.261602  79533.046980                 3   
276214  2529117   91880.835927  14378.758711  77502.077216                 3   
23033   1219396  122507.781236  46093.892095  76413.889142                 4   
306769  6265877    9698.532681  78146.856202  68448.323521                 4   
306766  6265869  127612.272121  60736.986051  66875.286070                 4   
307929  6272435  125060.026679  59018.503670  66041.523009                 4   
297595  4316103  109746.554024  43861.564332  65884.989692                 3   
279075  2558425  107194.308582  42101.533604  65092.774977                 4   
307907  6272347  127612.272121  62910.383173  64701.888948                 4   
286890  2700236  103110.715874  38720.346175  64390.369699                 3   

        YearMade  MachineHoursCurre

/tmp/ipython-input-4119580116.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_time_split['predictions'] = initial_val_preds_full_price
/tmp/ipython-input-4119580116.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_time_split['error'] = abs(val_time_split['SalePrice_adj'] - val_time_split['predictions'])


In [1]:
# --- שלב 0: ייבוא ספריות נוספות ---
from sklearn.ensemble import RandomForestClassifier

# ######################################################################
# ### שלב 7 (חדש): אימון מודל דו-שלבי ###
# ######################################################################
print("="*50)
print("\n--- שלב 7: מתחיל אימון מודל דו-שלבי (ממיין + מומחים) ---")

# 1. הגדרת סף למכונה "יקרה"
# נשתמש באחוזון ה-75 של המחירים המתוקננים כסף
price_threshold = df_train_processed['SalePrice_adj'].quantile(0.75)
print(f"סף המחיר למכונה 'יקרה' נקבע על: ${price_threshold:,.2f}")

# יצירת משתנה מטרה חדש לסיווג
df_train_processed['is_expensive'] = (df_train_processed['SalePrice_adj'] > price_threshold).astype(int)

# 2. הכנת נתונים לאימון
X_full_train = df_train_processed.drop(['SalePrice_Residual', 'SalePrice_adj', 'is_expensive'], axis=1)
y_full_train_residual = df_train_processed['SalePrice_Residual']
y_full_train_classifier = df_train_processed['is_expensive']


# 3. אימון הממיין (Classifier)
print("\nמאמן את המודל הממיין...")
classifier_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=15)
classifier_model.fit(X_full_train, y_full_train_classifier)
print("אימון הממיין הושלם.")


# 4. פיצול הנתונים ל"יקר" ו"לא יקר"
expensive_idx = df_train_processed[df_train_processed['is_expensive'] == 1].index
not_expensive_idx = df_train_processed[df_train_processed['is_expensive'] == 0].index

X_train_expensive = X_full_train.loc[expensive_idx]
y_train_expensive = y_full_train_residual.loc[expensive_idx]

X_train_not_expensive = X_full_train.loc[not_expensive_idx]
y_train_not_expensive = y_full_train_residual.loc[not_expensive_idx]


# 5. אימון המומחים (Regressors)
# נשתמש בפרמטרים המכוילים שלך או בפרמטרים טובים אחרים
params = {'n_jobs': -1, 'random_state': 42, 'n_estimators': 400, 'min_samples_leaf': 1, 'min_samples_split': 2, 'max_features': 0.5}

print("\nמאמן את המומחה למכונות יקרות...")
regressor_expensive = RandomForestRegressor(**params)
regressor_expensive.fit(X_train_expensive, y_train_expensive)
print("אימון המומחה למכונות יקרות הושלם.")

print("\nמאמן את המומחה למכונות רגילות...")
regressor_not_expensive = RandomForestRegressor(**params)
regressor_not_expensive.fit(X_train_not_expensive, y_train_not_expensive)
print("אימון המומחה למכונות רגילות הושלם.")


# ######################################################################
# ### שלב 8 (חדש): יצירת הגשה עם המודל הדו-שלבי ###
# ######################################################################
print("\n" + "="*50)
print("--- שלב 8: יוצר קובץ הגשה עם המודל הדו-שלבי ---")

# 1. התאמת עמודות
df_valid_processed_aligned = df_valid_processed[X_full_train.columns]

# 2. שימוש בממיין כדי להחליט לאיזה מומחה לשלוח כל מכונה
print("שלב 1/3: מסווג את נתוני המבחן ל'יקר' ו'לא יקר'...")
valid_classifications = classifier_model.predict(df_valid_processed_aligned)

# 3. חיזוי עם המומחה המתאים
print("שלב 2/3: מריץ חיזוי עם המומחים המתאימים...")
# יצירת מערך ריק לאחסון החיזויים
final_residuals_pred = np.zeros(len(df_valid_processed_aligned))

# זיהוי האינדקסים של כל קבוצה
expensive_mask = (valid_classifications == 1)
not_expensive_mask = (valid_classifications == 0)

# חיזוי לכל קבוצה בנפרד
final_residuals_pred[expensive_mask] = regressor_expensive.predict(df_valid_processed_aligned[expensive_mask])
final_residuals_pred[not_expensive_mask] = regressor_not_expensive.predict(df_valid_processed_aligned[not_expensive_mask])

# 4. שחזור המחיר הסופי (כמו קודם)
print("שלב 3/3: משחזר את המחיר הסופי...")
final_trend_values = df_valid_raw['SalePrice_Trend'].values
final_price_pred = final_trend_values + final_residuals_pred
final_price_pred_floored = np.maximum(final_price_pred, 1000)

# 5. יצירת ושמירת הקובץ
df_submission = pd.DataFrame({'SalesID': df_valid_raw['SalesID'], 'SalePrice': final_price_pred_floored})
submission_filename = f'{DRIVE_PATH}submission_TwoStage_model.csv'
df_submission.to_csv(submission_filename, index=False)

print(f"\nקובץ ההגשה '{submission_filename}' נשמר בהצלחה!")
print("5 השורות הראשונות בקובץ ההגשה:")
print(df_submission.head())


--- שלב 7: מתחיל אימון מודל דו-שלבי (ממיין + מומחים) ---


NameError: name 'df_train_processed' is not defined